# Classifiers {-}
## EE627 {-}
### Connor DePalma & Daniel Pelis - Ground Truth {-}

In [8]:
import numpy as np

In [9]:
with open('gbtSuggestions.csv','w') as f:
    row = 'TrackID,Predictor\n'
    f.write(row)

In [10]:
dataDir='./data/'
file_name_ratings=dataDir + 'output1.txt'
ratings = open(file_name_ratings, 'r')
with open(dataDir + 'gbt_weights.csv','r') as f:
    weights = f.readlines()[1]

In [11]:
weightList = [float(i) for i in weights.strip().split(',')]

In [12]:
weightList

[0.21206789182631944, 0.22481202380666993, 0.5631200843670104]

In [13]:
trackID_vec=[0]*6
albumRate_vec=[0]*6
artistRate_vec=[0]*6
genreRate_vec=[0]*6
lastUserID=-1
ii=0

In [14]:
for line in ratings:
    arr_rating=line.strip().split('|')
    userID= arr_rating[0]
    trackID= arr_rating[1]
    arr_rating = list(map(float, arr_rating))
    albumRate= arr_rating[2]
    artistRate=arr_rating[3]
    genreRate = 0
    if len(arr_rating) > 4:
        genreRate = np.mean(arr_rating[4:])
        
    if userID != lastUserID:
        ii=0

    trackID_vec[ii]=trackID
    albumRate_vec[ii]=albumRate
    artistRate_vec[ii]=artistRate
    genreRate_vec[ii] = genreRate
    
    ii=ii+1
    lastUserID=userID
    
    if ii == 6:
        total_ratings = {}
        for nn in range(0, 6):
            total_ratings[trackID_vec[nn]] = albumRate_vec[nn] * weightList[0]  + artistRate_vec[nn] * weightList[1] + genreRate_vec[nn] * weightList[2]

        total_ratings_sorted = total_ratings.copy()
        total_ratings_sorted = {k: v for k, v in sorted(total_ratings.items(), key=lambda item: item[1], reverse=True)}
        
        suggest = [0,0,0,1,1,1]
        for key in total_ratings_sorted.keys():
            total_ratings[key] = suggest.pop()
        
        with open('gbtSuggestions.csv','a') as f:
            for item_id, value in total_ratings.items():
                row = f'{userID}_{item_id},{value}\n'
                f.write(row)